In [3]:
import cv2
import json
import numpy as np
from typing import List, Tuple
from matplotlib import pyplot as plt
import pickle 
import os

def show_image(img, title=""):
    if len(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))

    plt.axis('off')
    if title != "":
        plt.title(title)
    plt.show()

In [4]:
IMGS_PATH = os.path.join(os.path.abspath("../../"), "dataset", "selected")

image = cv2.imread(os.path.join(IMGS_PATH, "0.png"))

In [5]:
with open('mask_rcnn/mrcnn_single_out.pickle', 'rb') as f:
    r = pickle.load(f)

In [6]:
# mask = r["masks"]

# splash = np.zeros_like(image)

# for i in range(mask.shape[-1]):
#     color = np.random.randint(0, 256, 3)  # Random RGB color
#     mask_single = mask[:, :, i]

#     # Convert boolean mask to uint8
#     mask_single = mask_single.astype(np.uint8)

#     # Create a color mask
#     color_mask = np.zeros_like(image)
#     color_mask[:, :, :] = color

#     # Apply color mask to the splash image
#     splash += cv2.bitwise_and(
#         color_mask, color_mask, mask=mask_single[:, :, np.newaxis]
#     )
#     break

# show_image(splash)

In [7]:
def group_contours(contours, max_distance):
    grouped_contours = []
    used = set()

    for i, contour1 in enumerate(contours):
        if i not in used:
            group = [contour1]
            used.add(i)

            for j, contour2 in enumerate(contours[i + 1:]):
                j = j + i + 1
                if j not in used:
                    M1 = cv2.moments(contour1)
                    centroid1 = (
                        int(M1["m10"] / M1["m00"]),
                        int(M1["m01"] / M1["m00"]),
                    )

                    M2 = cv2.moments(contour2)
                    centroid2 = (
                        int(M2["m10"] / M2["m00"]),
                        int(M2["m01"] / M2["m00"]),
                    )

                    distance = np.linalg.norm(
                        np.array(centroid1) - np.array(centroid2)
                    )

                    if distance < max_distance:
                        group.append(contour2)
                        used.add(j)

            grouped_contours.append(group)

    return grouped_contours

# r is a list of mask coming from mask-rcnn

mask = r["masks"]
scores = r["scores"]
roi = r["rois"]


# splash = np.zeros_like(image)
i_score = image.copy()
i_area = image.copy()
i_polygon = image.copy()
i_cont = image.copy()
i_ellipse = image.copy()

splash = image.copy()

SCORE_THRESHOLD = 0.7
AREA_THRESHOLD = 0.005
ELLIPSE_THRESHOLD = 0.05

# For some reasons the area difference even when they are very similar is quit
# huge, for this reason I'm applying this multiplier. I'm sure there's a reason
# for this issue, I just can't find it right now. I'm not using
# cv2.contourArea(polygons[0]) as it would add an additional stage of error.
AREA_MULTIPLIER = 4

IMAGE_AREA = image.shape[0] * image.shape[1]


SHOW = False
CROP = False
SAVE_CONT = False

dimensions = []
for i in range(mask.shape[-1]):
# for i in [0, 1, 90]:
# for i in [29, 50, 57]:
    # Select the current mask
    mask_single = mask[:, :, i]

    # Convert boolean mask to uint8
    mask_single = mask_single.astype(np.uint8) * 255
    area = np.sum(mask_single) * AREA_MULTIPLIER

    # Find contours using cv2.findContours
    contours, _ = cv2.findContours(
        mask_single, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if SHOW:
        if scores[i] < SCORE_THRESHOLD:
            cv2.drawContours(i_score, contours, -1, (0, 0, 255), 5)
        else:
            cv2.drawContours(i_score, contours, -1, (0, 255, 0), 5)

    if scores[i] < SCORE_THRESHOLD:
        continue

    if SHOW:
        if area < AREA_THRESHOLD * IMAGE_AREA:
            cv2.drawContours(i_area, contours, -1, (0, 0, 255), 5)
        else:
            cv2.drawContours(i_area, contours, -1, (0, 255, 0), 5)

    # Skip areas that are too small
    if area < AREA_THRESHOLD * IMAGE_AREA:
        continue

    # Convert contours to polygons
    polygons = [
        cv2.approxPolyDP(contour, 0.001 * cv2.arcLength(contour, True), True)
        for contour in contours
    ]

    if SHOW:
        if len(polygons) != 1:
            cv2.drawContours(i_polygon, contours, -1, (0, 0, 255), 5)
        else:
            cv2.drawContours(i_polygon, contours, -1, (0, 255, 0), 5)

    # If multiply polygons are found skip them
    if len(polygons) != 1:
        continue

    hull = cv2.convexHull(polygons[0])

    # Calculate ellipse
    ellipse_int = cv2.fitEllipse(hull)
    ellipse_ext = cv2.fitEllipse(polygons[0])

    major_axis_ext, minor_axis_ext = ellipse_ext[1]
    major_axis_int, minor_axis_int = ellipse_int[1]

    # Calculate the area of the ellipse
    ellipse_int_area = np.pi * major_axis_int * minor_axis_int
    ellipse_ext_area = np.pi * major_axis_ext * minor_axis_ext

    if SHOW:
        if (
            np.abs(ellipse_ext_area - ellipse_int_area)
            > np.mean([ellipse_ext_area, ellipse_int_area]) * ELLIPSE_THRESHOLD
        ):
            cv2.drawContours(splash, contours, -1, (0, 0, 255), 5)
        else:
            cv2.drawContours(splash, contours, -1, (0, 255, 0), 5)

    if (
        np.abs(ellipse_ext_area - ellipse_int_area)
        > np.mean([ellipse_ext_area, ellipse_int_area]) * ELLIPSE_THRESHOLD
    ):
        continue

    ### Find stuff inside the kiwi
    normalizedImg = np.zeros((800, 800))
    normalizedImg = cv2.normalize(image, normalizedImg, 0, 255, cv2.NORM_MINMAX)
    normalizedImg = image.copy()
    # Crop the image around the fruit
    c = cv2.bitwise_and(normalizedImg, cv2.cvtColor(mask_single, cv2.COLOR_GRAY2BGR))[
        roi[i][0]-10 : roi[i][2]+20, roi[i][1]-10 : roi[i][3]+10
    ]

    # Draw external contours in green and internal contours in blue
    # Convert the image to grayscale
    gray = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)

    
    kernel = np.ones((3, 3), np.uint8)

    # Perform opening (erosion followed by dilation)
    morph = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)

    # Apply GaussianBlur for smoothing
    blurred = cv2.GaussianBlur(morph, (5, 5), 0)

    edges = cv2.Canny(blurred, 100, 200)

    internal_contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    internal_contours = group_contours(internal_contours, 5)

    if SHOW:
        if (len(internal_contours)>1):
            cv2.drawContours(i_cont, contours, -1, (0, 0, 255), 5)
        else:
            cv2.drawContours(i_cont, contours, -1, (0, 255, 0), 5)

    if len(internal_contours)>1:
        continue

    # Memorize avg height and width (from the 2 ellipses)
    # mean_1, mean_2 = np.mean([major_axis_ext,major_axis_int]), np.mean([minor_axis_ext, minor_axis_int])
    mean_1, mean_2 = major_axis_int, minor_axis_int

    if SHOW:
        cv2.ellipse(i_ellipse, ellipse_ext, (0, 255, 0), 2)

    ##### Save Dimensions
    avg_height, avg_width = np.max([mean_1, mean_2]), np.min([mean_1, mean_2])
    dimensions.append((avg_height, avg_width))

    if SAVE_CONT:
        contour_image = np.zeros_like(c)

        for j in range(len(internal_contours)):
            color = (np.random.randint(0,256), np.random.randint(0,256), np.random.randint(0,256))
            
            cv2.drawContours(contour_image, internal_contours[j], -1, color ,1)

        # Display the original image and the image with contours
        plt.subplot(1,4,1)
        plt.imshow(cv2.cvtColor(c, cv2.COLOR_BGR2RGB))
        plt.title('Original image')
        
        plt.subplot(1, 4, 2)
        plt.imshow(cv2.cvtColor(blurred, cv2.COLOR_BGR2RGB))
        plt.title('kernel')

        plt.subplot(1, 4, 3)
        plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))
        plt.title('edges')

        plt.subplot(1, 4, 4)
        plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
        plt.title('Image with Contours')

        plt.savefig(f"../plot/single_selection/{i} {len(internal_contours)==1}.jpg")

if SHOW:
    size = 6
    fig, axes = plt.subplots(1, size, figsize=(50, 10))

    plt.subplot(1, size, 1)
    plt.title("Selected with scores")
    plt.axis("off")
    if CROP:
        plt.xlim([image.shape[0] // 2, image.shape[0]])
        plt.ylim([image.shape[1] // 2, image.shape[1]])
    plt.imshow(cv2.cvtColor(i_score, cv2.COLOR_BGR2RGB))

    plt.subplot(1, size, 2)
    plt.title("Selected with areas")
    plt.axis("off")
    if CROP:
        plt.xlim([image.shape[0] // 2, image.shape[0]])
        plt.ylim([image.shape[1] // 2, image.shape[1]])
    plt.imshow(cv2.cvtColor(i_area, cv2.COLOR_BGR2RGB))

    plt.subplot(1, size, 3)
    plt.title("Selected with polygons")
    plt.axis("off")
    if CROP:
        plt.xlim([image.shape[0] // 2, image.shape[0]])
        plt.ylim([image.shape[1] // 2, image.shape[1]])
    plt.imshow(cv2.cvtColor(i_polygon, cv2.COLOR_BGR2RGB))

    plt.subplot(1, size, 4)
    plt.title("Selected with ellipses")
    plt.axis("off")
    if CROP:
        plt.xlim([image.shape[0] // 2, image.shape[0]])
        plt.ylim([image.shape[1] // 2, image.shape[1]])
    plt.imshow(cv2.cvtColor(splash, cv2.COLOR_BGR2RGB))

    plt.subplot(1, size, 5)
    plt.title("Selected with internal contours")
    plt.axis("off")
    if CROP:
        plt.xlim([image.shape[0] // 2, image.shape[0]])
        plt.ylim([image.shape[1] // 2, image.shape[1]])
    plt.imshow(cv2.cvtColor(i_cont, cv2.COLOR_BGR2RGB))

    plt.subplot(1, size, 6)
    plt.title("Final ellipses")
    plt.axis("off")
    if CROP:
        plt.xlim([image.shape[0] // 2, image.shape[0]])
        plt.ylim([image.shape[1] // 2, image.shape[1]])
    plt.imshow(cv2.cvtColor(i_ellipse, cv2.COLOR_BGR2RGB))

    plt.suptitle("Mask selection workflow")
    plt.savefig(f"../plot/workflow.jpg", dpi=300)


In [12]:
with open("px_size.pickle", "wb+") as f:
    pickle.dump({"img_shape": image.shape,
                 "dimensions": dimensions},f)

In [55]:
# # r is a list of mask coming from mask-rcnn

# mask = r["masks"]
# scores = r["scores"]

# # splash = np.zeros_like(image)
# i_score = image.copy()
# i_area = image.copy()
# i_polygon = image.copy()

# splash = image.copy()

# SCORE_THRESHOLD = 0.7
# AREA_THRESHOLD = 0.005
# ELLIPSE_THRESHOLD = 0.05

# # For some reasons the area difference even when they are very similar is quit
# # huge, for this reason I'm applying this multiplier. I'm sure there's a reason
# # for this issue, I just can't find it right now. I'm not using
# # cv2.contourArea(polygons[0]) as it would add an additional stage of error.
# AREA_MULTIPLIER = 4

# IMAGE_AREA = image.shape[0] * image.shape[1]


# SHOW = True
# CROP = False
# a, b = 0, 0
# # for i in range(mask.shape[-1]):
# for i in [0,1,90]:
#     # i=90
#     # Select the current mask
#     mask_single = mask[:, :, i]

#     # Convert boolean mask to uint8
#     mask_single = mask_single.astype(np.uint8)
#     area = np.sum(mask_single) * AREA_MULTIPLIER

#     # Find contours using cv2.findContours
#     contours, _ = cv2.findContours(
#         mask_single, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
#     )

#     if SHOW:
#         if scores[i] < SCORE_THRESHOLD:
#             cv2.drawContours(i_score, contours, -1, (0, 0, 255), 5)
#         else:
#             cv2.drawContours(i_score, contours, -1, (0, 255, 0), 5)

#     if scores[i] < SCORE_THRESHOLD:
#         continue

#     if SHOW:
#         if area < AREA_THRESHOLD * IMAGE_AREA:
#             cv2.drawContours(i_area, contours, -1, (0, 0, 255), 5)
#         else:
#             cv2.drawContours(i_area, contours, -1, (0, 255, 0), 5)

#     # Skip areas that are too small
#     if area < AREA_THRESHOLD * IMAGE_AREA:
#         continue

#     # Convert contours to polygons
#     polygons = [
#         cv2.approxPolyDP(contour, 0.001 * cv2.arcLength(contour, True), True)
#         for contour in contours
#     ]

#     if SHOW:
#         if len(polygons) != 1:
#             cv2.drawContours(i_polygon, contours, -1, (0, 0, 255), 5)
#         else:
#             cv2.drawContours(i_polygon, contours, -1, (0, 255, 0), 5)

#     # If multiply polygons are found skip them
#     if len(polygons) != 1:
#         continue

#     hull = cv2.convexHull(polygons[0])

#     # Calculate ellipse
#     ellipse_int = cv2.fitEllipse(hull)
#     ellipse_ext = cv2.fitEllipse(polygons[0])

#     major_axis_ext, minor_axis_ext = ellipse_ext[1]
#     major_axis_int, minor_axis_int = ellipse_int[1]

#     # Calculate the area of the ellipse
#     ellipse_int_area = np.pi * major_axis_int * minor_axis_int
#     ellipse_ext_area = np.pi * major_axis_ext * minor_axis_ext

#     if SHOW:
#         if (
#             np.abs(ellipse_ext_area - ellipse_int_area)
#             > np.mean([ellipse_ext_area, ellipse_int_area]) * ELLIPSE_THRESHOLD
#         ):
#             cv2.drawContours(splash, contours, -1, (0, 0, 255), 5)
#         else:
#             cv2.drawContours(splash, contours, -1, (0, 255, 0), 5)

#     if (
#         np.abs(ellipse_ext_area - ellipse_int_area)
#         > np.mean([ellipse_ext_area, ellipse_int_area]) * ELLIPSE_THRESHOLD
#     ):
#         continue

    


# if SHOW:
#     size = 4
#     fig, axes = plt.subplots(1, size, figsize=(20, 10))

#     plt.subplot(1, size, 1)
#     plt.title("Selected with scores")
#     plt.axis("off")
#     if CROP:
#         plt.xlim([image.shape[0] // 2, image.shape[0]])
#         plt.ylim([image.shape[1] // 2, image.shape[1]])
#     plt.imshow(cv2.cvtColor(i_score, cv2.COLOR_BGR2RGB))

#     plt.subplot(1, size, 2)
#     plt.title("Selected with areas")
#     plt.axis("off")
#     if CROP:
#         plt.xlim([image.shape[0] // 2, image.shape[0]])
#         plt.ylim([image.shape[1] // 2, image.shape[1]])
#     plt.imshow(cv2.cvtColor(i_area, cv2.COLOR_BGR2RGB))

#     plt.subplot(1, size, 3)
#     plt.title("Selected with polygons")
#     plt.axis("off")
#     if CROP:
#         plt.xlim([image.shape[0] // 2, image.shape[0]])
#         plt.ylim([image.shape[1] // 2, image.shape[1]])
#     plt.imshow(cv2.cvtColor(i_polygon, cv2.COLOR_BGR2RGB))

#     plt.subplot(1, size, 4)
#     plt.title("Selected with ellipses")
#     plt.axis("off")
#     if CROP:
#         plt.xlim([image.shape[0] // 2, image.shape[0]])
#         plt.ylim([image.shape[1] // 2, image.shape[1]])
#     plt.imshow(cv2.cvtColor(splash, cv2.COLOR_BGR2RGB))

In [ ]:
if False:
        # Create an image for visualization
        visualization_img = np.zeros_like(image, dtype=np.uint8)

        # Draw polygons on the visualization image
        cv2.drawContours(visualization_img, polygons, -1, (255, 255, 255), 1)
        # Draw ellipse
        cv2.ellipse(visualization_img, ellipse_ext, (255, 0, 0), 1)
        cv2.ellipse(visualization_img, ellipse_int, (0, 255, 0), 1)

        # Visualize the original binary mask and the polygons
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(mask_single, cmap="gray")
        plt.title("Original Binary Mask")
        plt.xlim(
            [mask_single.shape[0] // 2, mask_single.shape[0]]
        )  # Set x-axis limits to show only the left half
        plt.ylim(
            [mask_single.shape[1] // 2, mask_single.shape[0]]
        )  # Set y-axis limits to show only the bottom half

        plt.subplot(1, 2, 2)
        plt.imshow(visualization_img, cmap="gray")
        plt.title("Polygons")
        plt.xlim(
            [visualization_img.shape[0] // 2, visualization_img.shape[0]]
        )  # Set x-axis limits to show only the left half
        plt.ylim(
            [visualization_img.shape[1] // 2, visualization_img.shape[1]]
        )  # Set y-axis limits to show only the bottom half

        # Create a larger figure with adjusted subplot spacing
        plt.subplots_adjust(wspace=0.5)  # Adjust the horizontal space between subplots
        plt.show()